# Question Answering Solution

### Introduction
- **Objective**: Develop and evaluate a question-answering model using the ConvFinQA dataset.

### Retrieval Technique
- **Approach**: Implement a retrieval-based method to find relevant information from the dataset. This involves using an embeddings index and context augmentation.

### Evaluation Metrics
- **Recall**: Measure the proportion of relevant information successfully retrieved by the model. Recall is calculated as the number of relevant documents retrieved divided by the total number of relevant documents in the dataset. This metric helps assess the effectiveness of the retrieval technique in finding all pertinent information needed to answer the questions accurately.


In [74]:
import json
from openai import OpenAI
import os
from dotenv import load_dotenv
import random
import pandas as pd
import re

In [2]:
with open("../../data/train_extended.json") as f:
    data = json.load(f)

#### Sampling 100 random entries for swiftness

In [3]:
df = pd.DataFrame(random.sample(data, 100))

In [4]:
df.head()

,pre_text,post_text,filename,table_ori,table,id,annotation,qa_0,qa_1,join_text,input_tokens,qa
0,[hii expects to incur higher costs to complete...,[.],HII/2011/page_90.pdf,"[[($ in millions), Employee Compensation, Othe...","[[( $ in millions ), employee compensation, ot...",Double_HII/2011/page_90.pdf,{'amt_table': '<table class='wikitable'><tr><t...,{'question': 'what is the net change in employ...,{'question': 'what is the net change in employ...,hii expects to incur higher costs to complete ...,1002,NaN
1,"[business subsequent to the acquisition ., the...",[rental expense for operating leases was appro...,LKQ/2018/page_102.pdf,"[[2019, $294,269], [2020, 256,172], [2021, 210...","[[2019, $ 294269], [2020, 256172], [2021, 2106...",Single_LKQ/2018/page_102.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,business subsequent to the acquisition .\nthe ...,919,{'question': 'what was the percentage change i...
2,"[item 6 ., selected financial data the followi...",[basic earnings ( loss ) per share ( 3 ) $ 2.9...,HII/2012/page_47.pdf,"[[, Year Ended December 31], [($ in millions, ...","[[( $ in millions except per share amounts ), ...",Single_HII/2012/page_47.pdf-1,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,item 6 .\nselected financial data the followin...,780,{'question': 'what was the net increase in tot...
3,[american tower corporation and subsidiaries n...,[aggregate rent expense ( including the effect...,AMT/2004/page_91.pdf,"[[2005, $106,116], [2006, 106,319], [2007, 106...","[[2005, $ 106116], [2006, 106319], [2007, 1060...",Single_AMT/2004/page_91.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,american tower corporation and subsidiaries no...,869,{'question': 'what was the average rental expe...
4,[backlog applied manufactures systems to meet ...,[applied 2019s backlog on any particular date ...,AMAT/2015/page_14.pdf,"[[, 2015, 2014, , (In millions, except percent...","[[, 2015, 2014, , ( in millions except percent...",Single_AMAT/2015/page_14.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,backlog applied manufactures systems to meet d...,600,{'question': 'what is the growth rate in the s...


In [5]:
len(df['qa'].dropna())

70

#### Will take one question per document.
#### Further questions beyond the first will be ignored.

In [6]:
df['qa'] = df['qa'].fillna(df['qa_0'])
df = df.drop(['qa_0', 'qa_1'], axis=1)
len(df['qa'].dropna())

100

### Client
Working with LMStudio we can use OpenAI's python library for local models too

In [7]:
# client = OpenAI(
#     api_key = "lm-studio",  
#     base_url="http://localhost:1234/v1"
# )

load_dotenv()

client = OpenAI(
    api_key = os.getenv('SECRET_OPENAI_API_KEY'), 
)

### Retrieval

In [8]:
from langchain_core.documents.base import Document

pages = []
for i, entry in enumerate(df['join_text']):
    doc_with_metadata = Document(
        page_content=(entry),
        metadata={"page": i})
    pages.append(doc_with_metadata)

In [9]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2")
# load it into Chroma
db = Chroma.from_documents(pages, embedding_function)

C:\Users\AntonioVelasco\AppData\Local\Temp\ipykernel_19268\3587671496.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(
c:\Users\AntonioVelasco\local_projects\tomoro\ConvFinQA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def context_augment(
                query,
                client,
                model="gpt-3.5-turbo",
                ):
    context_augmented = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": """
                        You are a helpful assistant that given a question, will produce a set of related terms to provide a better context for the retireval tool.
                        make sure to give just points being concise and only listing concepts
                        """
                },
                {
                    "role": "user",
                    "content": f"""
                        USER_QUERY: {query}
                        """,
                }
            ],
            model=model,
        ).choices[0].message.content
    return context_augmented

In [11]:
augmented_query = context_augment(df['qa'][0]['question'], client)
print(augmented_query)

- Employee compensation
- Net change
- 2011
- Financial report
- HR records


In [12]:
retrieved = db.similarity_search(df['qa'][i]['question'] + augmented_query, k=10)

In [13]:
# Function to extract numbers from text
def extract_number(text):
    text = text.replace(',', '')
    numbers = re.findall(r'\d+\.\d+|\d+', text)
    numbers = [float(num) for num in numbers]
    return numbers[0] if numbers else None

# Function to self-evaluate
def classifier(
                doc,
                query,
                client,
                model="gpt-3.5-turbo",
                verbose=False
                ):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a helpfull classifier that sorts context documents for a LLM
                Given a DOCUMENT and a USER_QUERY, you will reply with 1 if the content is relevant.
                Otherwise you will reply 0.
                """
            },
            {
                "role": "user",
                "content": f"""
                    USER_QUERY: {query}
                    DOCUMENT: {doc}
                    """,
            },
            {
                "role": "assistant",
                "content": f"""
                    Write exclusively a number as your output. Eg: 1
                    """,
            }
        ],
        model=model,
    ).choices[0].message.content
    if verbose:
        return response, extract_number(response)
    return extract_number(response)

In [14]:
classifier(retrieved[0].page_content, df['qa'][i]['question'], client)

0.0

In [16]:
retrieved

[Document(metadata={'page': 30}, page_content='management 2019s discussion and analysis of financial condition and results of operations 2013 ( continued ) ( amounts in millions , except per share amounts ) corporate and other expenses increased slightly during 2013 by $ 3.5 to $ 140.8 compared to 2012 , primarily due to an increase in salaries and related expenses , mainly attributable to higher base salaries , benefits and temporary help , partially offset by lower severance expenses and a decrease in office and general expenses .\nliquidity and capital resources cash flow overview the following tables summarize key financial data relating to our liquidity , capital resources and uses of capital. .\n\n| cash flow data | years ended december 31 , 2014 | years ended december 31 , 2013 | years ended december 31 , 2012 |\n| net income adjusted to reconcile net income to net cashprovided by operating activities1 | $ 831.2 | $ 598.4 | $ 697.2 |\n| net cash used in working capital b2 | -131

In [17]:
duplicates = []
retrieved_unique = []
for doc in retrieved:
    if doc.metadata['page'] not in duplicates:
        duplicates.append(doc.metadata['page'])
        retrieved_unique.append(doc)

In [18]:
retrieved_unique

[Document(metadata={'page': 30}, page_content='management 2019s discussion and analysis of financial condition and results of operations 2013 ( continued ) ( amounts in millions , except per share amounts ) corporate and other expenses increased slightly during 2013 by $ 3.5 to $ 140.8 compared to 2012 , primarily due to an increase in salaries and related expenses , mainly attributable to higher base salaries , benefits and temporary help , partially offset by lower severance expenses and a decrease in office and general expenses .\nliquidity and capital resources cash flow overview the following tables summarize key financial data relating to our liquidity , capital resources and uses of capital. .\n\n| cash flow data | years ended december 31 , 2014 | years ended december 31 , 2013 | years ended december 31 , 2012 |\n| net income adjusted to reconcile net income to net cashprovided by operating activities1 | $ 831.2 | $ 598.4 | $ 697.2 |\n| net cash used in working capital b2 | -131

In [19]:
binary_list = []
relevant_docs = []

for doc in retrieved_unique:
    binary = classifier(doc.page_content, df['qa'][doc.metadata['page']]['question'], client)
    binary_list.append(binary)
    if binary == 1:
        relevant_docs.append(doc)

In [20]:
binary_list

[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]

In [21]:
relevant_docs

[Document(metadata={'page': 30}, page_content='management 2019s discussion and analysis of financial condition and results of operations 2013 ( continued ) ( amounts in millions , except per share amounts ) corporate and other expenses increased slightly during 2013 by $ 3.5 to $ 140.8 compared to 2012 , primarily due to an increase in salaries and related expenses , mainly attributable to higher base salaries , benefits and temporary help , partially offset by lower severance expenses and a decrease in office and general expenses .\nliquidity and capital resources cash flow overview the following tables summarize key financial data relating to our liquidity , capital resources and uses of capital. .\n\n| cash flow data | years ended december 31 , 2014 | years ended december 31 , 2013 | years ended december 31 , 2012 |\n| net income adjusted to reconcile net income to net cashprovided by operating activities1 | $ 831.2 | $ 598.4 | $ 697.2 |\n| net cash used in working capital b2 | -131

Let's now evaluate the quality of the retrieval

In [27]:
doc_list = []

for i, entry in enumerate(df['qa']):
    augmented_query = context_augment(entry['question'], client)
    retrieved = db.similarity_search(entry['question'] + augmented_query, k=10)

    duplicates = []
    retrieved_unique = []
    for doc in retrieved:
        if doc.metadata['page'] not in duplicates:
            duplicates.append(doc.metadata['page'])
            retrieved_unique.append(doc)
    
    binary_list = []
    relevant_docs = []
    for doc in retrieved_unique:
        binary = classifier(doc.page_content, df['qa'][doc.metadata['page']]['question'], client)
        binary_list.append(binary)
        if binary == 1:
            relevant_docs.append(doc)
    doc_list.append(relevant_docs)
    
df['context'] = doc_list

In [29]:
df.head()

,pre_text,post_text,filename,table_ori,table,id,annotation,join_text,input_tokens,qa,context
0,[hii expects to incur higher costs to complete...,[.],HII/2011/page_90.pdf,"[[($ in millions), Employee Compensation, Othe...","[[( $ in millions ), employee compensation, ot...",Double_HII/2011/page_90.pdf,{'amt_table': '<table class='wikitable'><tr><t...,hii expects to incur higher costs to complete ...,1002,{'question': 'what is the net change in employ...,[page_content='table of contents interest expe...
1,"[business subsequent to the acquisition ., the...",[rental expense for operating leases was appro...,LKQ/2018/page_102.pdf,"[[2019, $294,269], [2020, 256,172], [2021, 210...","[[2019, $ 294269], [2020, 256172], [2021, 2106...",Single_LKQ/2018/page_102.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,business subsequent to the acquisition .\nthe ...,919,{'question': 'what was the percentage change i...,[page_content='operating expenses millions 201...
2,"[item 6 ., selected financial data the followi...",[basic earnings ( loss ) per share ( 3 ) $ 2.9...,HII/2012/page_47.pdf,"[[, Year Ended December 31], [($ in millions, ...","[[( $ in millions except per share amounts ), ...",Single_HII/2012/page_47.pdf-1,{'amt_table': '<table class='wikitable'><tr><t...,item 6 .\nselected financial data the followin...,780,{'question': 'what was the net increase in tot...,[page_content='consolidated income statement r...
3,[american tower corporation and subsidiaries n...,[aggregate rent expense ( including the effect...,AMT/2004/page_91.pdf,"[[2005, $106,116], [2006, 106,319], [2007, 106...","[[2005, $ 106116], [2006, 106319], [2007, 1060...",Single_AMT/2004/page_91.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,american tower corporation and subsidiaries no...,869,{'question': 'what was the average rental expe...,[page_content='wood products sales in the unit...
4,[backlog applied manufactures systems to meet ...,[applied 2019s backlog on any particular date ...,AMAT/2015/page_14.pdf,"[[, 2015, 2014, , (In millions, except percent...","[[, 2015, 2014, , ( in millions except percent...",Single_AMAT/2015/page_14.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,backlog applied manufactures systems to meet d...,600,{'question': 'what is the growth rate in the s...,"[page_content='as noted above , as a result of..."


In [30]:
df['context'][0]

[Document(metadata={'page': 41}, page_content='table of contents interest expense , net of capitalized interest increased $ 64 million , or 9.8% ( 9.8 % ) , to $ 710 million in 2013 from $ 646 million in 2012 primarily due to special charges of $ 92 million to recognize post-petition interest expense on unsecured obligations pursuant to the plan and penalty interest related to 10.5% ( 10.5 % ) secured notes and 7.50% ( 7.50 % ) senior secured notes .\nother nonoperating expense , net of $ 84 million in 2013 consists principally of net foreign currency losses of $ 55 million and early debt extinguishment charges of $ 48 million .\nother nonoperating income in 2012 consisted principally of a $ 280 million special credit related to the settlement of a commercial dispute partially offset by net foreign currency losses .\nreorganization items , net reorganization items refer to revenues , expenses ( including professional fees ) , realized gains and losses and provisions for losses that are

In [35]:
df1 = df.copy()

In [43]:
df1.head()

,pre_text,post_text,filename,table_ori,table,id,annotation,join_text,input_tokens,qa,context
0,[hii expects to incur higher costs to complete...,[.],HII/2011/page_90.pdf,"[[($ in millions), Employee Compensation, Othe...","[[( $ in millions ), employee compensation, ot...",Double_HII/2011/page_90.pdf,{'amt_table': '<table class='wikitable'><tr><t...,hii expects to incur higher costs to complete ...,1002,{'question': 'what is the net change in employ...,[page_content='table of contents interest expe...
1,"[business subsequent to the acquisition ., the...",[rental expense for operating leases was appro...,LKQ/2018/page_102.pdf,"[[2019, $294,269], [2020, 256,172], [2021, 210...","[[2019, $ 294269], [2020, 256172], [2021, 2106...",Single_LKQ/2018/page_102.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,business subsequent to the acquisition .\nthe ...,919,{'question': 'what was the percentage change i...,[page_content='operating expenses millions 201...
2,"[item 6 ., selected financial data the followi...",[basic earnings ( loss ) per share ( 3 ) $ 2.9...,HII/2012/page_47.pdf,"[[, Year Ended December 31], [($ in millions, ...","[[( $ in millions except per share amounts ), ...",Single_HII/2012/page_47.pdf-1,{'amt_table': '<table class='wikitable'><tr><t...,item 6 .\nselected financial data the followin...,780,{'question': 'what was the net increase in tot...,[page_content='consolidated income statement r...
3,[american tower corporation and subsidiaries n...,[aggregate rent expense ( including the effect...,AMT/2004/page_91.pdf,"[[2005, $106,116], [2006, 106,319], [2007, 106...","[[2005, $ 106116], [2006, 106319], [2007, 1060...",Single_AMT/2004/page_91.pdf-3,{'amt_table': '<table class='wikitable'><tr><t...,american tower corporation and subsidiaries no...,869,{'question': 'what was the average rental expe...,[page_content='wood products sales in the unit...
4,[backlog applied manufactures systems to meet ...,[applied 2019s backlog on any particular date ...,AMAT/2015/page_14.pdf,"[[, 2015, 2014, , (In millions, except percent...","[[, 2015, 2014, , ( in millions except percent...",Single_AMAT/2015/page_14.pdf-2,{'amt_table': '<table class='wikitable'><tr><t...,backlog applied manufactures systems to meet d...,600,{'question': 'what is the growth rate in the s...,"[page_content='as noted above , as a result of..."


In [68]:
df1['retrieved'] = df.apply(lambda row: [i.metadata['page'] for i in row['context']], axis=1)
df1['correct_retrieve'] = df1.apply(lambda row: row.name in row['retrieved'], axis=1)

In [71]:
# Calculate the percentage of True values
percentage_true = df1['correct_retrieve'].mean() * 100

print(f"Recall: {percentage_true:.2f}%")

Recall: 46.00%
